<a href="https://colab.research.google.com/github/DanielDLX/DLfinal/blob/master/bert_AGnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 导入包
# transformer提供了一些训练好的模型，可以很方便的使用。
!pip install transformers
import tensorflow as tf
import pandas as pd
import os
import tqdm
# 使用分类的模型，增加了一个head用于分类。
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import math
% matplotlib inline

tf.__version__

     |████████████████████████████████| 757kB 6.6MB/s 
     |████████████████████████████████| 1.1MB 19.3MB/s 
     |████████████████████████████████| 890kB 17.3MB/s 
     |████████████████████████████████| 3.0MB 42.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7b1320e0db27632c3287bcf8027da0c9b90f048ff830decd09b80f3a29b39493
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


'2.2.0'

In [ ]:
# 导入现成的模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4) # 分类类别数
model.summary()
model.config

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [ ]:
# 数据链接，可以在 https://course.fast.ai/datasets 找到。
# agnews数据集，类别，标题，描述。
ag_url = 'https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz'

In [ ]:
# 下载数据，并指定此时数据集的目录
ag_zip_file = tf.keras.utils.get_file(origin=ag_url,fname='ag_news_csv.tgz', extract=True)
base_dir = os.path.join(os.path.dirname(ag_zip_file), 'ag_news_csv')
os.listdir(base_dir)

11788288/11784419 [==============================] - 1s 0us/step


['classes.txt', 'readme.txt', 'test.csv', 'train.csv']

In [ ]:
# 读取数据
# base_dir = ''
train = pd.read_csv(os.path.join(base_dir, 'train.csv'), header=None)
print(len(train))
print(train.head())
test = pd.read_csv(os.path.join(base_dir, 'test.csv'), header=None)
print(len(test))
print(test.head())
f = open(os.path.join(base_dir, 'classes.txt'))
classes = f.readlines()
classes = [s.strip() for s in classes]
print(classes)

120000
   0  ...                                                  2
0  3  ...  Reuters - Short-sellers, Wall Street's dwindli...
1  3  ...  Reuters - Private investment firm Carlyle Grou...
2  3  ...  Reuters - Soaring crude prices plus worries\ab...
3  3  ...  Reuters - Authorities have halted oil export\f...
4  3  ...  AFP - Tearaway world oil prices, toppling reco...

[5 rows x 3 columns]
7600
   0  ...                                                  2
0  3  ...  Unions representing workers at Turner   Newall...
1  4  ...  SPACE.com - TORONTO, Canada -- A second\team o...
2  4  ...  AP - A company founded by a chemistry research...
3  4  ...  AP - It's barely dawn when Mike Fitzpatrick st...
4  4  ...  AP - Southern California's smog-fighting agenc...

[5 rows x 3 columns]
['World', 'Sports', 'Business', 'Sci/Tech']


In [ ]:
# transformers自带的tokenizer中的encoder会把一段文本进行编码，然后增加上CLS和SEP，其中CLS的id是101，SEP的编码是102,PAD是0。
# 所以  a   dog   is  not   a   table
# [cls]  a   dog   is  not   a   table  [sep]
# 101   1037  3899  2003 2025  1037  2795   102   0  0  0  ...  0 
%pprint #让列表横过来，好看一些。
tokenizer.encode(text='a dog is not a table', padding='max_length',max_length=512)[:20]

Pretty printing has been turned OFF


[101, 1037, 3899, 2003, 2025, 1037, 2795, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# 预处理数据
# 按照上面的例子把数据集中的文本进行分词处理，并且得到对应的labels。
max_length = 512
train_ids = [tokenizer.encode(text=sent, padding='max_length', max_length=max_length, return_tensors="tf") for sent in tqdm.notebook.tqdm(train[2])]
test_ids = [tokenizer.encode(text=sent, padding='max_length', max_length=max_length, return_tensors="tf") for sent in tqdm.notebook.tqdm(test[2])]
train_labels = train[0].values - 1
test_labels = test[0].values - 1

In [ ]:
# 把数据转成tensorflow张量
# train_ids是tf.Tensor组成得列表，所以用concat组合一下就行
train_ids = tf.concat(train_ids, 0)
# 把train_mask初始化为1，然后把train_ids等于0（PAD的部分）对应的值赋为0
train_mask = tf.ones(train_ids.shape)
train_mask = tf.where(tf.math.greater(train_ids, 0), train_mask, 0)
# labels本身是numpy数组，转为tf.Tensor
train_labels = tf.convert_to_tensor(train_labels)

# 测试集的处理同理test
test_ids = tf.concat(test_ids, 0)
test_mask = tf.ones(test_ids.shape)
test_mask = tf.where(tf.math.greater(test_ids, 0), test_mask, 0)
test_labels = tf.convert_to_tensor(test_labels)

In [ ]:
print(train_ids[0])
print(train_mask[0])
print(train_labels[0])

tf.Tensor(
[  101 26665  1011  2460  1011 19041  1010  2813  2395  1005  1055  1040
 11101  2989  1032  2316  1997 11087  1011 22330  8713  2015  1010  2024
  3773  2665  2153  1012   102     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0    

In [ ]:
# 训练参数
epochs = 1
batch_size = 4
validation_rate = 0.1

In [ ]:
# 模型编译
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [12]:
# 模型训练
model.fit(x=[train_ids, train_mask], 
     y=train_labels, 
     batch_size=batch_size, 
     epochs=epochs, 
     verbose=1, 
     callbacks=None,
     validation_split=validation_rate, 
     validation_data=None, 
     shuffle=True)

27000/27000 [==============================] - 8320s 308ms/step - loss: 0.3546 - accuracy: 0.8986 - val_loss: 0.5085 - val_accuracy: 0.8967


In [13]:
# 模型测试
model.evaluate(x=[test_ids, test_mask],
        y=test_labels, 
        batch_size=4, 
        verbose=1)

1900/1900 [==============================] - 178s 94ms/step - loss: 0.5331 - accuracy: 0.8979


[0.5330594778060913, 0.8978947401046753]